In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import dask.dataframe as ddf
from pandas import Series, DataFrame
%matplotlib inline

In [2]:
import seaborn as sn
import plotly.express as px

### Request for Census Tract Level Population data from Census API

In [3]:
stateCode = ['01', '02', '04', '05', '06', '08', '09', '10', '11', '12', '13', '15', '16',
             '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29',
            '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', 
             '44', '45', '46', '47', '48', '49', '50', '51', '53', '54', '55', '56']

In [4]:
len(stateCode)

51

In [5]:
import requests
import pandas as pd
from datetime import datetime
import time
import json
from io import StringIO

In [6]:
def collectAll_county(year, variables):
    basic_url = 'https://api.census.gov/data/' + year + '/acs/acs3?get=NAME,'+ variables + '&for=county:*&in=state:*&key=250f58e35283d400e4185b197e3abe9021079afe'
    r = requests.get(basic_url)
    x = r.json()
    print(x[0])
    variable_list = variables.split(',')
    column_list = ['county_name']
    for item in variable_list:
        column_list.append(item)
    column_list.append('state code')
    column_list.append('county code')
    df = pd.DataFrame(x[1:])
    df.columns = column_list
    return df

#### Total population: B01003_001E

In [7]:
year = '2008'
variables = 'B01003_001E'

In [8]:
pop_09 = collectAll_county(year,variables)
pop_09.head()

['NAME', 'B01003_001E', 'state', 'county']


,county_name,B01003_001E,state code,county code
0,"Riverside County, California",2055232,06,065
1,"Sacramento County, California",1380708,06,067
2,"San Benito County, California",54515,06,069
3,"San Bernardino County, California",1999753,06,071
4,"San Diego County, California",2965943,06,073


In [9]:
pop_09.shape

(1887, 4)

#### 1.per capita income: B19301_001E

In [10]:
variables = 'B19301_001E'
df1 = collectAll_county(year,variables)

['NAME', 'B19301_001E', 'state', 'county']


In [11]:
df1.shape

(1887, 4)

In [12]:
df1 = df1.rename(columns = {'B19301_001E': 'per_capita_income'})
df1['fips'] = df1['state code'] + df1['county code']
df1.head()

,county_name,per_capita_income,state code,county code,fips
0,"Fresno County, California",20640,06,019,06019
1,"Glenn County, California",18754,06,021,06021
2,"Humboldt County, California",23262,06,023,06023
3,"Imperial County, California",16043,06,025,06025
4,"Kern County, California",20410,06,029,06029


#### 2.Percent of Population Below Poverty Level: (C17002_002E+C17002_003E)/C17002_001E

In [13]:
variables = 'C17002_002E,C17002_003E,C17002_001E'
df2 = collectAll_county(year,variables)

['NAME', 'C17002_002E', 'C17002_003E', 'C17002_001E', 'state', 'county']


In [14]:
df2['C17002_002E'] = df2['C17002_002E'].astype('int')
df2['C17002_003E'] = df2['C17002_003E'].astype('int')
df2['C17002_001E'] = df2['C17002_001E'].astype('int')

In [15]:
df2['percent_below_poverty'] = ((df2['C17002_002E'] + df2['C17002_003E'])/df2['C17002_001E'])*100

In [16]:
df2['fips'] = df2['state code'] + df2['county code']
df2.head()

,county_name,C17002_002E,C17002_003E,C17002_001E,state code,county code,percent_below_poverty,fips
0,"Broomfield County, Colorado",1808,2393,53139,08,014,7.905681,08014
1,"Delta County, Colorado",1334,2116,29572,08,029,11.666441,08029
2,"Denver County, Colorado",48668,58777,575484,08,031,18.670371,08031
3,"Douglas County, Colorado",3922,3277,268855,08,035,2.677652,08035
4,"Eagle County, Colorado",2634,2349,50564,08,037,9.854837,08037


In [17]:
df2.shape

(1887, 8)

#### 3.Percent of Population 25+ with Less than a 12th Grade Education: (B15002_003E,B15002_004E,B15002_005E,B15002_003E,B15002_006E,B15002_007E,B15002_008E,B15002_009E,B15002_010E,B15002_020E,B15002_021E,B15002_022E,B15002_023E,B15002_024E,B15002_025E,B15002_026E,B15002_027E)/B15003_001E

In [18]:
def collectAll_county2(year, variables):
    basic_url = 'https://api.census.gov/data/' + year + '/acs/acs1?get=NAME,'+ variables + '&for=county:*&in=state:*'
    r = requests.get(basic_url)
    x = r.json()
    print(x[0])
    variable_list = variables.split(',')
    column_list = ['county_name']
    for item in variable_list:
        column_list.append(item)
    column_list.append('state code')
    column_list.append('county code')
    df = pd.DataFrame(x[1:])
    df.columns = column_list
    return df

In [19]:
variables = 'B15002_003E,B15002_004E,B15002_005E,B15002_006E,B15002_007E,B15002_008E,B15002_009E,B15002_010E,B15002_020E,B15002_021E,B15002_022E,B15002_023E,B15002_024E,B15002_025E,B15002_026E,B15002_027E,B15002_001E'
df3 = collectAll_county(year,variables)

['NAME', 'B15002_003E', 'B15002_004E', 'B15002_005E', 'B15002_006E', 'B15002_007E', 'B15002_008E', 'B15002_009E', 'B15002_010E', 'B15002_020E', 'B15002_021E', 'B15002_022E', 'B15002_023E', 'B15002_024E', 'B15002_025E', 'B15002_026E', 'B15002_027E', 'B15002_001E', 'state', 'county']


In [20]:
df3.shape

(1887, 20)

In [21]:
def changeInteger(value):
    if value != None:
        return int(value)
    else:
        return np.nan

In [22]:
df3['B15002_003E'] = df3['B15002_003E'].apply(changeInteger)
df3['B15002_004E'] = df3['B15002_004E'].apply(changeInteger)
df3['B15002_005E'] = df3['B15002_005E'].apply(changeInteger)
df3['B15002_006E'] = df3['B15002_006E'].apply(changeInteger)
df3['B15002_007E'] = df3['B15002_007E'].apply(changeInteger)
df3['B15002_008E'] = df3['B15002_008E'].apply(changeInteger)
df3['B15002_009E'] = df3['B15002_009E'].apply(changeInteger)
df3['B15002_010E'] = df3['B15002_010E'].apply(changeInteger)
df3['B15002_020E'] = df3['B15002_020E'].apply(changeInteger)
df3['B15002_021E'] = df3['B15002_021E'].apply(changeInteger)
df3['B15002_022E'] = df3['B15002_022E'].apply(changeInteger)
df3['B15002_023E'] = df3['B15002_023E'].apply(changeInteger)
df3['B15002_024E'] = df3['B15002_024E'].apply(changeInteger)
df3['B15002_025E'] = df3['B15002_025E'].apply(changeInteger)
df3['B15002_026E'] = df3['B15002_026E'].apply(changeInteger)
df3['B15002_027E'] = df3['B15002_027E'].apply(changeInteger)
df3['B15002_001E'] = df3['B15002_001E'].apply(changeInteger)

In [23]:
df3['percent_no_highschool_diploma'] = ((df3['B15002_003E']+df3['B15002_004E']+df3['B15002_005E']+df3['B15002_006E']+df3['B15002_007E']+df3['B15002_008E']+df3['B15002_009E']
                                       +df3['B15002_010E']+df3['B15002_020E']+df3['B15002_021E']+df3['B15002_022E']+df3['B15002_023E']+df3['B15002_024E']+df3['B15002_025E']+df3['B15002_003E']
                                       +df3['B15002_026E']+df3['B15002_027E'])/df3['B15002_001E'])*100

In [24]:
df3['fips'] = df3['state code'] + df3['county code']
df3.head()

,county_name,B15002_003E,B15002_004E,B15002_005E,B15002_006E,B15002_007E,B15002_008E,B15002_009E,B15002_010E,B15002_020E,...,B15002_023E,B15002_024E,B15002_025E,B15002_026E,B15002_027E,B15002_001E,state code,county code,percent_no_highschool_diploma,fips
0,"Franklin County, Maine",66.0,0.0,31.0,268.0,154.0,498.0,326.0,180.0,10.0,...,421.0,151.0,236.0,247.0,160.0,20392.0,23,007,13.828953,23007
1,"Hancock County, Maine",45.0,0.0,55.0,388.0,448.0,330.0,355.0,268.0,38.0,...,284.0,258.0,414.0,311.0,172.0,38160.0,23,009,9.174528,23009
2,"Kennebec County, Maine",84.0,91.0,133.0,1062.0,734.0,943.0,698.0,612.0,166.0,...,958.0,640.0,596.0,768.0,581.0,84655.0,23,011,10.059654,23011
3,"Knox County, Maine",0.0,164.0,60.0,639.0,276.0,303.0,287.0,244.0,17.0,...,310.0,326.0,417.0,202.0,163.0,29860.0,23,013,11.413262,23013
4,"Lincoln County, Maine",23.0,15.0,145.0,217.0,150.0,137.0,317.0,184.0,10.0,...,247.0,129.0,226.0,61.0,97.0,25657.0,23,015,7.806836,23015


#### 4.Percent of Population Living in Mobile Homes: (B25033_006E+B25033_007E+B25033_012E+B25033_013E)/B25033_001E

In [25]:
variables = 'B25033_006E,B25033_007E,B25033_012E,B25033_013E,B25033_001E'
df4 = collectAll_county(year,variables)

['NAME', 'B25033_006E', 'B25033_007E', 'B25033_012E', 'B25033_013E', 'B25033_001E', 'state', 'county']


In [26]:
df4.shape

(1887, 8)

In [27]:
df4['B25033_006E'] = df4['B25033_006E'].apply(changeInteger)
df4['B25033_007E'] = df4['B25033_007E'].apply(changeInteger)
df4['B25033_012E'] = df4['B25033_012E'].apply(changeInteger)
df4['B25033_013E'] = df4['B25033_013E'].apply(changeInteger)
df4['B25033_001E'] = df4['B25033_001E'].apply(changeInteger)

In [28]:
df4['percent_mobile_homes'] = ((df4['B25033_006E'] + df4['B25033_007E'] + df4['B25033_012E'] + df4['B25033_013E'])/df4['B25033_001E'])*100

In [29]:
df4['fips'] = df4['state code'] + df4['county code']
df4.head()

,county_name,B25033_006E,B25033_007E,B25033_012E,B25033_013E,B25033_001E,state code,county code,percent_mobile_homes,fips
0,"Autauga County, Alabama",7434.0,72.0,3093.0,0.0,49744.0,01,001,21.307092,01001
1,"Baldwin County, Alabama",17793.0,214.0,8544.0,126.0,169739.0,01,003,15.716482,01003
2,"Barbour County, Alabama",5608.0,0.0,2088.0,0.0,26568.0,01,005,28.967179,01005
3,"Bibb County, Alabama",5614.0,0.0,906.0,0.0,20769.0,01,007,31.392941,01007
4,"Blount County, Alabama",12031.0,0.0,3864.0,0.0,56024.0,01,009,28.371769,01009


#### 5.Percent of Population with No Vehicle Available: (B25044_003E+B25044_010E)/B25044_001E

In [30]:
variables = 'B25044_003E,B25044_010E,B25044_001E'
df5 = collectAll_county(year,variables)

['NAME', 'B25044_003E', 'B25044_010E', 'B25044_001E', 'state', 'county']


In [31]:
df5.shape

(1887, 6)

In [32]:
df5['B25044_003E'] = df5['B25044_003E'].apply(changeInteger)
df5['B25044_010E'] = df5['B25044_010E'].apply(changeInteger)
df5['B25044_001E'] = df5['B25044_001E'].apply(changeInteger)

In [33]:
df5['percent_no_vehicle'] = ((df5['B25044_003E'] + df5['B25044_010E'])/df5['B25044_001E'])*100

In [34]:
df5['fips'] = df5['state code'] + df5['county code']
df5.head()

,county_name,B25044_003E,B25044_010E,B25044_001E,state code,county code,percent_no_vehicle,fips
0,"Conway County, Arkansas",326,351,8183,05,029,8.273249,05029
1,"Craighead County, Arkansas",524,1708,34872,05,031,6.400551,05031
2,"Crawford County, Arkansas",435,761,21254,05,033,5.627176,05033
3,"Crittenden County, Arkansas",370,1541,20102,05,035,9.506517,05035
4,"Faulkner County, Arkansas",323,1333,39462,05,045,4.196442,05045


#### 6.Percent of Population Unemployed:sum(B21005_006E, B21005_007E, B21005_011E,B21005_012E,B21005_017E,B21005_022E,B21005_028E,B21005_029E,B21005_033E,B21005_034E)/B21005_001E

In [35]:
variables = 'B21005_006E,B21005_007E,B21005_011E,B21005_012E,B21005_017E,B21005_022E,B21005_028E,B21005_029E,B21005_033E,B21005_034E,B21005_001E'
df6 = collectAll_county2(year,variables)

['NAME', 'B21005_006E', 'B21005_007E', 'B21005_011E', 'B21005_012E', 'B21005_017E', 'B21005_022E', 'B21005_028E', 'B21005_029E', 'B21005_033E', 'B21005_034E', 'B21005_001E', 'state', 'county']


In [36]:
df6.shape

(802, 14)

In [37]:
df6['B21005_006E'] = df6['B21005_006E'].apply(changeInteger)
df6['B21005_007E'] = df6['B21005_007E'].apply(changeInteger)
df6['B21005_011E'] = df6['B21005_011E'].apply(changeInteger)
df6['B21005_012E'] = df6['B21005_012E'].apply(changeInteger)
df6['B21005_017E'] = df6['B21005_017E'].apply(changeInteger)
df6['B21005_022E'] = df6['B21005_022E'].apply(changeInteger)
df6['B21005_028E'] = df6['B21005_028E'].apply(changeInteger)
df6['B21005_029E'] = df6['B21005_029E'].apply(changeInteger)
df6['B21005_033E'] = df6['B21005_033E'].apply(changeInteger)
df6['B21005_034E'] = df6['B21005_034E'].apply(changeInteger)
df6['B21005_001E'] = df6['B21005_001E'].apply(changeInteger)

In [38]:
df6['percent_unemployed_CDC'] = ((df6['B21005_006E']+df6['B21005_007E']+df6['B21005_011E']+df6['B21005_012E']+df6['B21005_017E']+df6['B21005_022E']+df6['B21005_028E']+df6['B21005_029E']+df6['B21005_033E']+df6['B21005_034E'])/df6['B21005_001E'])*100

In [39]:
df6['fips'] = df6['state code'] + df6['county code']
df6.head()

,county_name,B21005_006E,B21005_007E,B21005_011E,B21005_012E,B21005_017E,B21005_022E,B21005_028E,B21005_029E,B21005_033E,B21005_034E,B21005_001E,state code,county code,percent_unemployed_CDC,fips
0,"Grayson County, Texas",0,27,1765,6754,0,765,302,937,360,4407,72839,48,181,21.028570,48181
1,"Gregg County, Texas",151,259,1330,5832,0,598,0,840,135,4010,70480,48,183,18.664869,48183
2,"Guadalupe County, Texas",113,0,2090,5490,98,1140,167,1122,58,2981,71335,48,187,18.586949,48187
3,"Harris County, Texas",1743,2437,54418,205789,1339,36692,1207,14752,7776,106583,2521031,48,201,17.165041,48201
4,"Hays County, Texas",0,0,4101,12156,422,694,0,351,637,3670,101391,48,209,21.728753,48209


#### 7.Percent of Population Living In Accommodations with Less Than 1 Room Per Person/Crowding: (B25014_005E+B25014_006E+B25014_007E+B25014_011E+B25014_012E+B25014_013E)/B25014_001E

In [40]:
variables = 'B25014_005E,B25014_006E,B25014_007E,B25014_011E,B25014_012E,B25014_013E,B25014_001E'
df7 = collectAll_county(year,variables)

['NAME', 'B25014_005E', 'B25014_006E', 'B25014_007E', 'B25014_011E', 'B25014_012E', 'B25014_013E', 'B25014_001E', 'state', 'county']


In [41]:
df7.shape

(1887, 10)

In [42]:
df7['B25014_005E'] = df7['B25014_005E'].apply(changeInteger)
df7['B25014_006E'] = df7['B25014_006E'].apply(changeInteger)
df7['B25014_007E'] = df7['B25014_007E'].apply(changeInteger)
df7['B25014_011E'] = df7['B25014_011E'].apply(changeInteger)
df7['B25014_012E'] = df7['B25014_012E'].apply(changeInteger)
df7['B25014_013E'] = df7['B25014_013E'].apply(changeInteger)
df7['B25014_001E'] = df7['B25014_001E'].apply(changeInteger)

In [43]:
df7['percent_overcrowding'] = ((df7['B25014_005E']+df7['B25014_006E']+df7['B25014_007E']+df7['B25014_011E']+df7['B25014_012E']+df7['B25014_013E'])/df7['B25014_001E'])*100

In [44]:
df7['fips'] = df7['state code'] + df7['county code']
df7.head()

,county_name,B25014_005E,B25014_006E,B25014_007E,B25014_011E,B25014_012E,B25014_013E,B25014_001E,state code,county code,percent_overcrowding,fips
0,"Boone County, Arkansas",126.0,0.0,9.0,78.0,85.0,0.0,13766.0,05,009,2.164754,05009
1,"Carroll County, Arkansas",101.0,52.0,50.0,152.0,0.0,13.0,11019.0,05,015,3.339686,05015
2,"Columbia County, Arkansas",39.0,14.0,0.0,107.0,20.0,0.0,10378.0,05,027,1.734438,05027
3,"Craighead County, Arkansas",263.0,0.0,0.0,414.0,33.0,0.0,34872.0,05,031,2.036017,05031
4,"Crawford County, Arkansas",227.0,37.0,0.0,98.0,0.0,31.0,21254.0,05,033,1.849064,05033


#### 8.Percent of Housing Units with 10+ Units in Structure: (B25024_007E+B25024_008E+B25024_009E)/B25024_001E

In [45]:
variables = 'B25024_007E,B25024_008E,B25024_009E,B25024_001E'
df8 = collectAll_county(year,variables)

['NAME', 'B25024_007E', 'B25024_008E', 'B25024_009E', 'B25024_001E', 'state', 'county']


In [46]:
df8.shape

(1887, 7)

In [47]:
df8['B25024_007E'] = df8['B25024_007E'].apply(changeInteger)
df8['B25024_008E'] = df8['B25024_008E'].apply(changeInteger)
df8['B25024_009E'] = df8['B25024_009E'].apply(changeInteger)
df8['B25024_001E'] = df8['B25024_001E'].apply(changeInteger)

In [48]:
df8['percent_multi_unit_housing'] = ((df8['B25024_007E']+df8['B25024_008E']+df8['B25024_009E'])/df8['B25024_001E'])*100

In [49]:
df8['fips'] = df8['state code'] + df8['county code']
df8.head()

,county_name,B25024_007E,B25024_008E,B25024_009E,B25024_001E,state code,county code,percent_multi_unit_housing,fips
0,"Orange County, California",58041.0,43073.0,85412.0,1029310.0,06,059,18.121460,06059
1,"Placer County, California",4168.0,1756.0,3851.0,144813.0,06,061,6.750085,06061
2,"Plumas County, California",110.0,110.0,0.0,14900.0,06,063,1.476510,06063
3,"Riverside County, California",26144.0,13547.0,17684.0,754415.0,06,065,7.605231,06065
4,"Sacramento County, California",29550.0,13711.0,23097.0,547187.0,06,067,12.127116,06067


#### 9.Percent Of Population 65+: (B01001_020E,B01001_021E,B01001_022E,B01001_023E,B01001_024E,B01001_025E,B01001_044E,B01001_045E,B01001_046E,B01001_047E,B01001_049E)/B01001_001E

In [50]:
variables = 'B01001_020E,B01001_021E,B01001_022E,B01001_023E,B01001_024E,B01001_025E,B01001_044E,B01001_045E,B01001_046E,B01001_047E,B01001_048E,B01001_049E,B01001_001E'
df9 = collectAll_county(year,variables)

['NAME', 'B01001_020E', 'B01001_021E', 'B01001_022E', 'B01001_023E', 'B01001_024E', 'B01001_025E', 'B01001_044E', 'B01001_045E', 'B01001_046E', 'B01001_047E', 'B01001_048E', 'B01001_049E', 'B01001_001E', 'state', 'county']


In [51]:
df9.shape

(1887, 16)

In [52]:
df9['B01001_020E'] = df9['B01001_020E'].apply(changeInteger)
df9['B01001_021E'] = df9['B01001_021E'].apply(changeInteger)
df9['B01001_022E'] = df9['B01001_022E'].apply(changeInteger)
df9['B01001_023E'] = df9['B01001_023E'].apply(changeInteger)
df9['B01001_024E'] = df9['B01001_024E'].apply(changeInteger)
df9['B01001_025E'] = df9['B01001_025E'].apply(changeInteger)
df9['B01001_044E'] = df9['B01001_044E'].apply(changeInteger)
df9['B01001_045E'] = df9['B01001_045E'].apply(changeInteger)
df9['B01001_046E'] = df9['B01001_046E'].apply(changeInteger)
df9['B01001_047E'] = df9['B01001_047E'].apply(changeInteger)
df9['B01001_048E'] = df9['B01001_048E'].apply(changeInteger)
df9['B01001_049E'] = df9['B01001_049E'].apply(changeInteger)
df9['B01001_001E'] = df9['B01001_001E'].apply(changeInteger)

In [53]:
df9['percent_age_65_and_older'] = ((df9['B01001_020E']+df9['B01001_021E']+df9['B01001_022E']+df9['B01001_023E']+df9['B01001_024E']+df9['B01001_025E']+df9['B01001_044E']
                                  +df9['B01001_045E']+df9['B01001_046E']+df9['B01001_047E']+df9['B01001_048E']+df9['B01001_049E'])/df9['B01001_001E'])*100

In [54]:
df9['fips'] = df9['state code'] + df9['county code']
df9.head()

,county_name,B01001_020E,B01001_021E,B01001_022E,B01001_023E,B01001_024E,B01001_025E,B01001_044E,B01001_045E,B01001_046E,B01001_047E,B01001_048E,B01001_049E,B01001_001E,state code,county code,percent_age_65_and_older,fips
0,"Franklin County, Maine",218,365,530,481,137,204,242,347,592,295,254,648,29882,23,007,14.433438,23007
1,"Hancock County, Maine",573,731,770,763,461,490,461,825,1036,781,775,966,53183,23,009,16.230750,23009
2,"Kennebec County, Maine",963,1386,1887,1526,1061,692,1200,1625,2005,2132,1714,1537,120711,23,011,14.686317,23011
3,"Knox County, Maine",408,627,581,696,364,317,419,640,961,843,941,428,40738,23,013,17.735284,23013
4,"Lincoln County, Maine",259,470,866,612,497,281,500,531,723,623,485,771,34730,23,015,19.055572,23015


#### 10.Percent Of Population Who Do Not Speak English: B99163_005E/B99163_001E

In [55]:
variables = 'B99163_005E,B99163_001E'
df10 = collectAll_county(year,variables)

['NAME', 'B99163_005E', 'B99163_001E', 'state', 'county']


In [56]:
df10.shape

(1887, 5)

In [57]:
df10['B99163_005E'] = df10['B99163_005E'].apply(changeInteger)
df10['B99163_001E'] = df10['B99163_001E'].apply(changeInteger)

In [58]:
df10['percent_limited_english_abilities'] = (df10['B99163_005E']/df10['B99163_001E'])*100

In [59]:
df10['fips'] = df10['state code'] + df10['county code']
df10.head()

,county_name,B99163_005E,B99163_001E,state code,county code,percent_limited_english_abilities,fips
0,"Crittenden County, Arkansas",1242,47771,05,035,2.599904,05035
1,"Faulkner County, Arkansas",3598,97030,05,045,3.708132,05045
2,"Garland County, Arkansas",5874,90502,05,051,6.490464,05051
3,"Greene County, Arkansas",667,37578,05,055,1.774975,05055
4,"Hempstead County, Arkansas",2833,21442,05,057,13.212387,05057


#### 11.Percent of Population 17 Years of Age and Under: (B01001_003E+B01001_004E+B01001_005E+B01001_006E+B01001_027E+B01001_028E+B01001_029E+B01001_030E)/B01003_001E

In [60]:
variables = 'B01001_003E,B01001_004E,B01001_005E,B01001_006E,B01001_027E,B01001_028E,B01001_029E,B01001_030E,B01003_001E'
df11 = collectAll_county(year,variables)

['NAME', 'B01001_003E', 'B01001_004E', 'B01001_005E', 'B01001_006E', 'B01001_027E', 'B01001_028E', 'B01001_029E', 'B01001_030E', 'B01003_001E', 'state', 'county']


In [61]:
df11.shape

(1887, 12)

In [62]:
df11['B01001_003E'] = df11['B01001_003E'].apply(changeInteger)
df11['B01001_004E'] = df11['B01001_004E'].apply(changeInteger)
df11['B01001_005E'] = df11['B01001_005E'].apply(changeInteger)
df11['B01001_006E'] = df11['B01001_006E'].apply(changeInteger)
df11['B01001_027E'] = df11['B01001_027E'].apply(changeInteger)
df11['B01001_028E'] = df11['B01001_028E'].apply(changeInteger)
df11['B01001_029E'] = df11['B01001_029E'].apply(changeInteger)
df11['B01001_030E'] = df11['B01001_030E'].apply(changeInteger)
df11['B01003_001E'] = df11['B01003_001E'].apply(changeInteger)

In [63]:
df11['percent_age_17_and_younger'] = ((df11['B01001_003E']+df11['B01001_004E']+df11['B01001_005E']+df11['B01001_006E']+df11['B01001_027E']+df11['B01001_028E']+df11['B01001_029E']+df11['B01001_030E'])/df11['B01003_001E'])*100

In [64]:
df11['fips'] = df11['state code'] + df11['county code']
df11.head()

,county_name,B01001_003E,B01001_004E,B01001_005E,B01001_006E,B01001_027E,B01001_028E,B01001_029E,B01001_030E,B01003_001E,state code,county code,percent_age_17_and_younger,fips
0,"Riverside County, California",82856,76398,84355,50823,79538,73076,80693,48522,2055232,06,065,28.038732,06065
1,"Sacramento County, California",52453,48543,51495,32498,50426,46707,48998,30883,1380708,06,067,26.218650,06067
2,"San Benito County, California",2288,2149,2609,1446,2030,2199,2145,1354,54515,06,069,29.753279,06069
3,"San Bernardino County, California",83516,80231,85536,54791,79748,75661,82010,52226,1999753,06,071,29.689617,06071
4,"San Diego County, California",113491,100541,101251,63850,108672,93891,98241,60409,2965943,06,073,24.961572,06073


#### 12.Percent Minority: 1-(B03002_003E/B03002_001E)

In [65]:
variables = 'B03002_003E,B03002_001E'
df12 = collectAll_county(year,variables)

['NAME', 'B03002_003E', 'B03002_001E', 'state', 'county']


In [66]:
df12.shape

(1887, 5)

In [67]:
df12['B03002_003E'] = df12['B03002_003E'].apply(changeInteger)
df12['B03002_001E'] = df12['B03002_001E'].apply(changeInteger)

In [68]:
df12['percent_minorities'] = (1 - (df12['B03002_003E']/df12['B03002_001E']))*100

In [69]:
df12['fips'] = df12['state code'] + df12['county code']
df12.head()

,county_name,B03002_003E,B03002_001E,state code,county code,percent_minorities,fips
0,"Berrien County, Michigan",124583.0,159398.0,26,021,21.841554,26021
1,"Seward County, Kansas",9181.0,22839.0,20,175,59.801217,20175
2,"Shawnee County, Kansas",134765.0,173387.0,20,177,22.275026,20177
3,"Wyandotte County, Kansas",73065.0,153698.0,20,209,52.461971,20209
4,"Christian County, Kentucky",56012.0,79774.0,21,047,29.786647,21047


#### 13.Percent of Population with a Disability: (B18101_004E+B18101_007E+B18101_010E+B18101_013E+B18101_016E+B18101_019E)/B18101_002E + (B18101_023E+B18101_026E+B18101_029E+B18101_032E+B18101_035E+B18101_038E)/B18101_021E

In [70]:
variables = 'B18101_004E,B18101_007E,B18101_010E,B18101_013E,B18101_016E,B18101_019E,B18101_002E,B18101_023E,B18101_026E,B18101_029E,B18101_032E,B18101_035E,B18101_038E,B18101_021E'
df13 = collectAll_county2(year,variables)

['NAME', 'B18101_004E', 'B18101_007E', 'B18101_010E', 'B18101_013E', 'B18101_016E', 'B18101_019E', 'B18101_002E', 'B18101_023E', 'B18101_026E', 'B18101_029E', 'B18101_032E', 'B18101_035E', 'B18101_038E', 'B18101_021E', 'state', 'county']


In [71]:
df13.shape

(802, 17)

In [72]:
df13['B18101_004E'] = df13['B18101_004E'].apply(changeInteger)
df13['B18101_007E'] = df13['B18101_007E'].apply(changeInteger)
df13['B18101_010E'] = df13['B18101_010E'].apply(changeInteger)
df13['B18101_013E'] = df13['B18101_013E'].apply(changeInteger)
df13['B18101_016E'] = df13['B18101_016E'].apply(changeInteger)
df13['B18101_002E'] = df13['B18101_002E'].apply(changeInteger)
df13['B18101_023E'] = df13['B18101_023E'].apply(changeInteger)
df13['B18101_026E'] = df13['B18101_026E'].apply(changeInteger)
df13['B18101_029E'] = df13['B18101_029E'].apply(changeInteger)
df13['B18101_032E'] = df13['B18101_032E'].apply(changeInteger)
df13['B18101_035E'] = df13['B18101_035E'].apply(changeInteger)
df13['B18101_038E'] = df13['B18101_038E'].apply(changeInteger)
df13['B18101_021E'] = df13['B18101_021E'].apply(changeInteger)

In [73]:
df13['percent_disabled'] = ((df13['B18101_004E']+df13['B18101_007E']+df13['B18101_010E']+df13['B18101_013E']+df13['B18101_016E'])/df13['B18101_002E'] + (df13['B18101_023E']+df13['B18101_026E']+df13['B18101_029E']+df13['B18101_032E']+df13['B18101_035E']+df13['B18101_038E'])/df13['B18101_021E'])*100

In [74]:
df13['fips'] = df13['state code'] + df13['county code']
df13.head()

,county_name,B18101_004E,B18101_007E,B18101_010E,B18101_013E,B18101_016E,B18101_019E,B18101_002E,B18101_023E,B18101_026E,B18101_029E,B18101_032E,B18101_035E,B18101_038E,B18101_021E,state code,county code,percent_disabled,fips
0,"Grayson County, Texas",64,505,903,2866,1401,2016,56846,0,11,391,3550,1342,2347,59893,48,181,22.853448,48181
1,"Gregg County, Texas",95,599,555,2930,1025,1451,55825,0,600,1542,4636,1628,2264,60096,48,183,27.076914,48183
2,"Guadalupe County, Texas",0,1423,1125,4246,951,1259,56868,0,679,1151,3486,1131,2490,58194,48,187,28.976512,48187
3,"Harris County, Texas",419,20943,23977,72643,21701,23467,1983997,969,10548,18673,81461,29730,44680,1978224,48,201,16.445941,48201
4,"Hays County, Texas",0,740,1779,2626,911,1692,74630,0,231,1335,2838,938,2231,73322,48,209,18.443114,48209


#### 14.Percent of Population Living in Group Quarters: B26001_001E/B01003_001E

In [75]:
variables = 'B26001_001E,B01003_001E'
df14 = collectAll_county(year,variables)

['NAME', 'B26001_001E', 'B01003_001E', 'state', 'county']


In [76]:
df14.shape

(1887, 5)

In [77]:
df14['B26001_001E'] = df14['B26001_001E'].apply(changeInteger)
df14['B01003_001E'] = df14['B01003_001E'].apply(changeInteger)

In [78]:
df14['percent_institutionalized_in_group_quarters'] = (df14['B26001_001E']/df14['B01003_001E'])*100

In [79]:
df14['fips'] = df14['state code'] + df14['county code']
df14.head()

,county_name,B26001_001E,B01003_001E,state code,county code,percent_institutionalized_in_group_quarters,fips
0,"Riverside County, California",28744.0,2055232,06,065,1.398577,06065
1,"Sacramento County, California",28671.0,1380708,06,067,2.076543,06067
2,"San Benito County, California",NaN,54515,06,069,NaN,06069
3,"San Bernardino County, California",44193.0,1999753,06,071,2.209923,06071
4,"San Diego County, California",111740.0,2965943,06,073,3.767436,06073


#### 15.Percent of Children Living in Single Parent Households: (B09008_010E+B09008_011E+B09008_012E)/B09008_001E

In [80]:
variables = 'B09008_010E,B09008_011E,B09008_012E,B09008_001E'
df15 = collectAll_county2(year,variables)

['NAME', 'B09008_010E', 'B09008_011E', 'B09008_012E', 'B09008_001E', 'state', 'county']


In [81]:
df15.shape

(802, 7)

In [82]:
df15['B09008_010E'] = df15['B09008_010E'].apply(changeInteger)
df15['B09008_011E'] = df15['B09008_011E'].apply(changeInteger)
df15['B09008_012E'] = df15['B09008_012E'].apply(changeInteger)
df15['B09008_001E'] = df15['B09008_001E'].apply(changeInteger)

In [83]:
df15['percent_single_parent_households_CDC'] = ((df15['B09008_010E'] + df15['B09008_011E']+df15['B09008_012E'])/df15['B09008_001E'])*100

In [84]:
df15['fips'] = df15['state code'] + df15['county code']
df15.head()

,county_name,B09008_010E,B09008_011E,B09008_012E,B09008_001E,state code,county code,percent_single_parent_households_CDC,fips
0,"Litchfield County, Connecticut",1102.0,4423.0,51.0,39769.0,09,005,14.020971,09005
1,"Middlesex County, Connecticut",415.0,6776.0,0.0,35704.0,09,007,20.140600,09007
2,"New Haven County, Connecticut",5379.0,47671.0,263.0,194392.0,09,009,27.425511,09009
3,"New London County, Connecticut",1539.0,11976.0,0.0,60013.0,09,011,22.520121,09011
4,"Tolland County, Connecticut",791.0,4827.0,0.0,29375.0,09,013,19.125106,09013


### age 18 to 34
#### (B01001_007E+B01001_008E+B01001_009E+B01001_010E+B01001_011E+B01001_012E+B01001_031E+B01001_032E+B01001_033E+B01001_034E+B01001_035E+B01001_036E)/B01001_001E

In [85]:
variables = 'B01001_007E,B01001_008E,B01001_009E,B01001_010E,B01001_011E,B01001_012E,B01001_031E,B01001_032E,B01001_033E,B01001_034E,B01001_035E,B01001_036E,B01001_001E'
df16 = collectAll_county(year,variables)

['NAME', 'B01001_007E', 'B01001_008E', 'B01001_009E', 'B01001_010E', 'B01001_011E', 'B01001_012E', 'B01001_031E', 'B01001_032E', 'B01001_033E', 'B01001_034E', 'B01001_035E', 'B01001_036E', 'B01001_001E', 'state', 'county']


In [86]:
df16['B01001_007E'] = df16['B01001_007E'].apply(changeInteger)
df16['B01001_008E'] = df16['B01001_008E'].apply(changeInteger)
df16['B01001_009E'] = df16['B01001_009E'].apply(changeInteger)
df16['B01001_010E'] = df16['B01001_010E'].apply(changeInteger)
df16['B01001_011E'] = df16['B01001_011E'].apply(changeInteger)
df16['B01001_012E'] = df16['B01001_012E'].apply(changeInteger)
df16['B01001_031E'] = df16['B01001_031E'].apply(changeInteger)
df16['B01001_032E'] = df16['B01001_032E'].apply(changeInteger)
df16['B01001_033E'] = df16['B01001_033E'].apply(changeInteger)
df16['B01001_034E'] = df16['B01001_034E'].apply(changeInteger)
df16['B01001_035E'] = df16['B01001_035E'].apply(changeInteger)
df16['B01001_036E'] = df16['B01001_036E'].apply(changeInteger)
df16['B01001_001E'] = df16['B01001_001E'].apply(changeInteger)

In [87]:
df16['percent_age_18_to_34'] = ((df16['B01001_007E'] + df16['B01001_008E'] +df16['B01001_009E'] +df16['B01001_010E'] +df16['B01001_011E']+df16['B01001_012E']+df16['B01001_031E']+df16['B01001_032E']+df16['B01001_033E']+df16['B01001_034E']+df16['B01001_035E']+df16['B01001_036E'])/df16['B01001_001E'])*100

In [88]:
df16['fips'] = df16['state code'] + df16['county code']
df16.head()

,county_name,B01001_007E,B01001_008E,B01001_009E,B01001_010E,B01001_011E,B01001_012E,B01001_031E,B01001_032E,B01001_033E,B01001_034E,B01001_035E,B01001_036E,B01001_001E,state code,county code,percent_age_18_to_34,fips
0,"Franklin County, Maine",469,401,175,529,949,701,763,313,441,490,1009,834,29882,23,007,23.673114,23007
1,"Hancock County, Maine",708,488,304,934,1341,1338,622,391,322,710,1410,1417,53183,23,009,18.774796,23009
2,"Kennebec County, Maine",1727,1014,626,1999,3697,3151,1656,927,803,2007,3936,3435,120711,23,011,20.692398,23011
3,"Knox County, Maine",376,255,179,643,1164,1216,383,190,435,362,1077,1098,40738,23,013,18.110855,23013
4,"Lincoln County, Maine",340,177,189,477,949,805,292,254,234,327,923,901,34730,23,015,16.896055,23015


In [89]:
df16.shape

(1887, 18)

### Age 35 to 44
#### (B01001_013E+B01001_014E+B01001_037E+B01001_038E)/B01001_001E

In [90]:
variables = 'B01001_013E,B01001_014E,B01001_037E,B01001_038E,B01001_001E'
df17 = collectAll_county(year,variables)

['NAME', 'B01001_013E', 'B01001_014E', 'B01001_037E', 'B01001_038E', 'B01001_001E', 'state', 'county']


In [91]:
df17['B01001_013E'] = df17['B01001_013E'].apply(changeInteger)
df17['B01001_014E'] = df17['B01001_014E'].apply(changeInteger)
df17['B01001_037E'] = df17['B01001_037E'].apply(changeInteger)
df17['B01001_038E'] = df17['B01001_038E'].apply(changeInteger)
df17['B01001_001E'] = df17['B01001_001E'].apply(changeInteger)

In [92]:
df17['percent_age_35_to_44'] = ((df17['B01001_013E']+df17['B01001_014E'] +df17['B01001_037E']+df17['B01001_038E'])/df17['B01001_001E'])*100

In [93]:
df17['fips'] = df17['state code'] + df17['county code']
df17.head()

,county_name,B01001_013E,B01001_014E,B01001_037E,B01001_038E,B01001_001E,state code,county code,percent_age_35_to_44,fips
0,"Franklin County, Maine",1004,890,1009,943,29882,23,007,12.870624,23007
1,"Hancock County, Maine",1419,2060,1967,1754,53183,23,009,13.538161,23009
2,"Kennebec County, Maine",3713,4569,4063,4456,120711,23,011,13.918367,23011
3,"Knox County, Maine",1239,1676,1311,1358,40738,23,013,13.707104,23013
4,"Lincoln County, Maine",1042,1135,1076,1273,34730,23,015,13.031961,23015


In [94]:
df17.shape

(1887, 10)

### age 45 to 54
#### (B01001_015E+B01001_016E+B01001_039E+B01001_040E)/B01001_001E

In [95]:
variables = 'B01001_015E,B01001_016E,B01001_039E,B01001_040E,B01001_001E'
df18 = collectAll_county(year,variables)

['NAME', 'B01001_015E', 'B01001_016E', 'B01001_039E', 'B01001_040E', 'B01001_001E', 'state', 'county']


In [96]:
df18['B01001_015E'] = df18['B01001_015E'].apply(changeInteger)
df18['B01001_016E'] = df18['B01001_016E'].apply(changeInteger)
df18['B01001_039E'] = df18['B01001_039E'].apply(changeInteger)
df18['B01001_040E'] = df18['B01001_040E'].apply(changeInteger)
df18['B01001_001E'] = df18['B01001_001E'].apply(changeInteger)

In [97]:
df18['percent_age_45_to_54'] = ((df18['B01001_015E']+df18['B01001_016E'] +df18['B01001_039E']+df18['B01001_040E'])/df18['B01001_001E'])*100

In [98]:
df18['fips'] = df18['state code'] + df18['county code']
df18.head()

,county_name,B01001_015E,B01001_016E,B01001_039E,B01001_040E,B01001_001E,state code,county code,percent_age_45_to_54,fips
0,"Franklin County, Maine",1188,1119,1159,1196,29882,23,007,15.601365,23007
1,"Hancock County, Maine",2109,2215,2292,2332,53183,23,009,16.824925,23009
2,"Kennebec County, Maine",4854,4841,5491,5069,120711,23,011,16.779747,23011
3,"Knox County, Maine",1606,1703,1652,1783,40738,23,013,16.554568,23013
4,"Lincoln County, Maine",1393,1400,1439,1451,34730,23,015,16.363375,23015


In [99]:
df18.shape

(1887, 10)

### age 55 to 64
#### (B01001_017E+B01001_018E+B01001_019E+B01001_041E+B01001_042E+B01001_043E)/B01001_001E

In [100]:
variables = 'B01001_017E,B01001_018E,B01001_019E,B01001_041E,B01001_042E,B01001_043E,B01001_001E'
df19 = collectAll_county(year,variables)

['NAME', 'B01001_017E', 'B01001_018E', 'B01001_019E', 'B01001_041E', 'B01001_042E', 'B01001_043E', 'B01001_001E', 'state', 'county']


In [101]:
df19['B01001_017E'] = df19['B01001_017E'].apply(changeInteger)
df19['B01001_018E'] = df19['B01001_018E'].apply(changeInteger)
df19['B01001_019E'] = df19['B01001_019E'].apply(changeInteger)
df19['B01001_041E'] = df19['B01001_041E'].apply(changeInteger)
df19['B01001_042E'] = df19['B01001_042E'].apply(changeInteger)
df19['B01001_043E'] = df19['B01001_043E'].apply(changeInteger)
df19['B01001_001E'] = df19['B01001_001E'].apply(changeInteger)

In [102]:
df19['percent_age_55_to_64'] = ((df19['B01001_017E']+df19['B01001_018E'] +df19['B01001_019E']+df19['B01001_041E']+df19['B01001_042E']+df19['B01001_043E'])/df19['B01001_001E'])*100

In [103]:
df19['fips'] = df19['state code'] + df19['county code']
df19.head()

,county_name,B01001_017E,B01001_018E,B01001_019E,B01001_041E,B01001_042E,B01001_043E,B01001_001E,state code,county code,percent_age_55_to_64,fips
0,"Franklin County, Maine",1050,432,563,947,439,647,29882,23,007,13.647012,23007
1,"Hancock County, Maine",2202,726,871,2304,775,996,53183,23,009,14.805483,23009
2,"Kennebec County, Maine",4502,1399,1816,4216,1695,2024,120711,23,011,12.966507,23011
3,"Knox County, Maine",1426,463,939,1595,555,774,40738,23,013,14.119495,23013
4,"Lincoln County, Maine",1146,615,806,1185,546,954,34730,23,015,15.122373,23015


In [104]:
df19.shape

(1887, 12)

### Combine 19 dataframes together

In [105]:
df = df1[['fips','per_capita_income']].merge(df2[['fips', 'percent_below_poverty']], on = 'fips', how = 'left')

In [106]:
df = df.merge(df3[['fips', 'percent_no_highschool_diploma']], on = 'fips', how = 'left')
df = df.merge(df4[['fips', 'percent_mobile_homes']], on = 'fips', how = 'left')
df = df.merge(df5[['fips', 'percent_no_vehicle']], on = 'fips', how = 'left')
df = df.merge(df7[['fips', 'percent_overcrowding']], on = 'fips', how = 'left')
df = df.merge(df8[['fips', 'percent_multi_unit_housing']], on = 'fips', how = 'left')
df = df.merge(df9[['fips', 'percent_age_65_and_older']], on = 'fips', how = 'left')
df = df.merge(df10[['fips', 'percent_limited_english_abilities']], on = 'fips', how = 'left')
df = df.merge(df12[['fips', 'percent_minorities']], on = 'fips', how = 'left')
df = df.merge(df14[['fips', 'percent_institutionalized_in_group_quarters']], on = 'fips', how = 'left')
df = df.merge(df16[['fips', 'percent_age_18_to_34']], on = 'fips', how = 'left')
df = df.merge(df17[['fips', 'percent_age_35_to_44']], on = 'fips', how = 'left')
df = df.merge(df18[['fips', 'percent_age_45_to_54']], on = 'fips', how = 'left')
df = df.merge(df19[['fips', 'percent_age_55_to_64']], on = 'fips', how = 'left')
df.head()

,fips,per_capita_income,percent_below_poverty,percent_no_highschool_diploma,percent_mobile_homes,percent_no_vehicle,percent_overcrowding,percent_multi_unit_housing,percent_age_65_and_older,percent_limited_english_abilities,percent_minorities,percent_institutionalized_in_group_quarters,percent_age_18_to_34,percent_age_35_to_44,percent_age_45_to_54,percent_age_55_to_64
0,06019,20640,21.229365,29.072699,3.625817,8.690495,9.245634,7.380423,9.768282,39.844420,64.285866,2.091568,26.490886,13.014809,12.227972,8.518278
1,06021,18754,18.980373,27.965513,12.288287,5.725111,4.504315,2.765580,11.945258,33.522132,41.849496,NaN,25.062531,12.642035,13.399557,9.418995
2,06023,23262,18.370465,11.249824,8.647616,6.123264,3.338406,4.386680,12.593776,8.682825,20.644390,3.377891,28.808273,11.735727,14.229967,12.080188
3,06025,16043,21.175196,38.440408,8.481997,10.955716,9.139551,10.351045,10.494396,69.066877,83.423412,6.424035,26.715442,13.416563,12.261519,7.823163
4,06029,20410,19.907261,30.492072,6.159056,7.511524,8.837356,4.498301,8.884501,39.231084,58.276640,5.136185,27.424286,13.549665,12.329109,7.814462


In [107]:
df.head()

,fips,per_capita_income,percent_below_poverty,percent_no_highschool_diploma,percent_mobile_homes,percent_no_vehicle,percent_overcrowding,percent_multi_unit_housing,percent_age_65_and_older,percent_limited_english_abilities,percent_minorities,percent_institutionalized_in_group_quarters,percent_age_18_to_34,percent_age_35_to_44,percent_age_45_to_54,percent_age_55_to_64
0,06019,20640,21.229365,29.072699,3.625817,8.690495,9.245634,7.380423,9.768282,39.844420,64.285866,2.091568,26.490886,13.014809,12.227972,8.518278
1,06021,18754,18.980373,27.965513,12.288287,5.725111,4.504315,2.765580,11.945258,33.522132,41.849496,NaN,25.062531,12.642035,13.399557,9.418995
2,06023,23262,18.370465,11.249824,8.647616,6.123264,3.338406,4.386680,12.593776,8.682825,20.644390,3.377891,28.808273,11.735727,14.229967,12.080188
3,06025,16043,21.175196,38.440408,8.481997,10.955716,9.139551,10.351045,10.494396,69.066877,83.423412,6.424035,26.715442,13.416563,12.261519,7.823163
4,06029,20410,19.907261,30.492072,6.159056,7.511524,8.837356,4.498301,8.884501,39.231084,58.276640,5.136185,27.424286,13.549665,12.329109,7.814462


In [108]:
df = df.sort_values('fips')
df.head()

,fips,per_capita_income,percent_below_poverty,percent_no_highschool_diploma,percent_mobile_homes,percent_no_vehicle,percent_overcrowding,percent_multi_unit_housing,percent_age_65_and_older,percent_limited_english_abilities,percent_minorities,percent_institutionalized_in_group_quarters,percent_age_18_to_34,percent_age_35_to_44,percent_age_45_to_54,percent_age_55_to_64
1757,01001,24051,11.455588,16.100346,21.307092,5.252272,1.741686,1.330231,10.990270,3.660903,21.910180,NaN,21.970489,14.998794,14.831940,10.570119
1758,01003,26678,10.347473,12.585528,15.716482,3.883075,1.893972,14.317360,16.512392,4.170307,14.768412,0.996226,21.232218,13.325984,14.164144,11.308451
1759,01005,15605,25.389379,28.331032,28.967179,10.202337,2.374846,1.017871,12.984760,3.663809,NaN,9.620357,23.785549,14.491768,13.722956,12.066268
1760,01007,21064,13.039694,27.623892,31.392941,6.716991,NaN,2.034884,12.494761,3.110490,NaN,NaN,22.390910,17.035347,13.146742,11.209426
1761,01009,20463,11.986534,27.343238,28.371769,4.088000,3.876461,0.344479,13.946950,5.855445,NaN,0.930150,21.978780,14.010610,14.201592,11.306808


In [109]:
df.to_csv('SDoH_3years/Social_Economic_3years_2008.csv')